In [ ]:
!pip install -U keras==2.2.4
!pip install matplotlib
!pip install dlr

In [ ]:
import tensorflow as tf
from sagemaker.tensorflow import TensorFlow
from tensorflow.keras.datasets import mnist
import numpy as np
from matplotlib import pyplot as plt
import keras
import sagemaker, os
from time import sleep
from uuid import uuid4
import dlr
print(keras.__version__)
print(tf.__version__)

In [ ]:
work_dir = './work'
!mkdir -p {work_dir}

TRAIN_X_PATH = os.path.join(work_dir,'train_X.npy')
TEST_X_PATH = os.path.join(work_dir,'test_X.npy')
TRAIN_Y_PATH = os.path.join(work_dir,'train_y.npy')
TEST_Y_PATH = os.path.join(work_dir,'test_y.npy')
(train_X, train_y), (test_X, test_y) = mnist.load_data()
train_X = (train_X-127.5)/127.5
test_X = (test_X-127.5)/127.5
# channel last
train_X = train_X.reshape((train_X.shape[0],train_X.shape[1],train_X.shape[2],1))
test_X = test_X.reshape((test_X.shape[0],test_X.shape[1],test_X.shape[2],1))
# one-hot
train_y = np.eye(10)[train_y]
test_y = np.eye(10)[test_y]
np.save(TRAIN_X_PATH,train_X)
np.save(TEST_X_PATH,test_X)
np.save(TRAIN_Y_PATH,train_y)
np.save(TEST_Y_PATH,test_y)

In [ ]:
# check shapes
print(train_X.shape,train_y.shape,test_X.shape,test_y.shape)

In [ ]:
role = sagemaker.get_execution_role()
sess = sagemaker.session.Session()
train_X_uri = sess.upload_data(path=TRAIN_X_PATH, bucket=sess.default_bucket(), key_prefix='sagemaker/mnist')
train_y_uri = sess.upload_data(path=TRAIN_Y_PATH, bucket=sess.default_bucket(), key_prefix='sagemaker/mnist')
test_X_uri = sess.upload_data(path=TEST_X_PATH, bucket=sess.default_bucket(), key_prefix='sagemaker/mnist')
test_y_uri = sess.upload_data(path=TEST_Y_PATH, bucket=sess.default_bucket(), key_prefix='sagemaker/mnist')
print(train_X_uri)
print(train_y_uri)
print(test_X_uri)
print(test_y_uri)

## Classifier Train

In [ ]:
estimator = TensorFlow(
    entry_point='./src/classifier_train.py',
    role=role,
    instance_count=1,
    instance_type='ml.g4dn.xlarge',
    framework_version='1.13', # SageMaker Neo を使うために keras 2.2.4 に合わせる
    py_version='py3',
    hyperparameters={
        'epochs':30,
        'increment':'False'
    }
)

In [ ]:
%%time

print(train_X_uri[:-11]) # dir以下全て
estimator.fit({
    'train': train_X_uri[:-11],
})

In [ ]:
classifier_model_uri = estimator.latest_training_job.describe()['ModelArtifacts']['S3ModelArtifacts']
print(classifier_model_uri)

## Neo Compile

In [ ]:
import boto3
sm_client = boto3.client('sagemaker')

In [ ]:
classifier_output_s3_location = f's3://{sagemaker.session.Session().default_bucket()}/sagemaker/mnist/classifier/compilied_model'
print(classifier_output_s3_location)

In [ ]:
classifier_compile_jobname = f'classifier-{str(uuid4())}'
print(classifier_compile_jobname)
response = sm_client.create_compilation_job(
    CompilationJobName=classifier_compile_jobname,
    RoleArn=role,
    InputConfig={
        'S3Uri': classifier_model_uri,
        'DataInputConfig': '{"input_1":[1,1,28,28]}',
        'Framework': 'KERAS',
    },
    OutputConfig={
        'S3OutputLocation': classifier_output_s3_location,
        'TargetDevice': 'ml_c5',
    },
    StoppingCondition={
        'MaxRuntimeInSeconds': 900,
        'MaxWaitTimeInSeconds': 900
    },
)

In [ ]:
while True:
    response = sm_client.describe_compilation_job(CompilationJobName=classifier_compile_jobname)
    status = response['CompilationJobStatus']
    if status in ['COMPLETED','FAILED','STOPPED'] : 
        print('!')
        print(status)
        classifier_neo_model_uri = response['ModelArtifacts']['S3ModelArtifacts']
        break
    else:
        print('.',end='')
    sleep(5)
print(classifier_neo_model_uri)

### ファイルの配置

In [ ]:
publisher_dir = './src/ggv2/components/artifacts/com.example.Publisher/1.0.0'
subscriber_dir = './src/ggv2/components/artifacts/com.example.Subscriber/1.0.0'
iotpublisher_classifier_dir = './src/ggv2/components/artifacts/com.example.IoTPublisher/1.0.1/classifier'

# test_x の配置
!mkdir -p {publisher_dir}
!mv {work_dir}/test_X.npy {publisher_dir}/test_X.npy

# # classifier1.h5 のダウンロード・解凍・gz削除
!aws s3 cp {classifier_model_uri} {work_dir}/
!mkdir -p {subscriber_dir}
!tar zxvf {work_dir}/model.tar.gz -C {subscriber_dir}
!rm {work_dir}/model.tar.gz 

# Neo コンパイル済モデルのダウンロード・解凍・gz削除

!aws s3 cp {classifier_neo_model_uri} {work_dir}/
!mkdir -p {iotpublisher_classifier_dir}
!tar zxvf {work_dir}/model-ml_c5.tar.gz -C {iotpublisher_classifier_dir}
!rm {work_dir}/model-ml_c5.tar.gz

## 動作確認
### keras model

In [ ]:
classifier = keras.models.load_model(f'{subscriber_dir}/classifier.h5')

In [ ]:
classifier.summary()

In [ ]:
pred_y = classifier.predict(test_X[0:1,:,:,:])
plt.imshow(test_X[0,:,:,0],'gray')

In [ ]:
np.argmax(pred_y[0,:])

## Neo Model

In [ ]:
import dlr
import numpy as np

classifier_neo = dlr.DLRModel(iotpublisher_classifier_dir, 'cpu', 0)

In [ ]:
pred_y = classifier_neo.run(test_X[0,:,:,:].reshape(1,1,28,28))[0]
np.argmax(pred_y)